In [53]:
import numpy as np
import random
import plotly.graph_objs as go
from plotly import offline
import time

In [54]:
publicity_rate_to_single_pull_rate = 12.395 / 4.877

focus_regular_weapon_rate = 0.02479 / publicity_rate_to_single_pull_rate
focus_regular_stigmata_rate = 0.01240 / publicity_rate_to_single_pull_rate
focus_regular_4_star_exclude_up_rate = 0.12395 / publicity_rate_to_single_pull_rate - focus_regular_weapon_rate - focus_regular_stigmata_rate * 3.0
expansion_regular_weapon_rate = 0.01831 / publicity_rate_to_single_pull_rate
expansion_regular_stigmata_rate = 0.00916 / publicity_rate_to_single_pull_rate
expansion_regular_4_star_exclude_up_rate = 0.12395 / publicity_rate_to_single_pull_rate - expansion_regular_weapon_rate - expansion_regular_stigmata_rate * 3.0
new_regular_weapon_rate = 0.02479 / publicity_rate_to_single_pull_rate
new_regular_stigmata_rate = 0.01240 / publicity_rate_to_single_pull_rate
new_regular_4_star_exclude_up_rate = 0.12395 / publicity_rate_to_single_pull_rate - new_regular_weapon_rate - new_regular_stigmata_rate * 3.0

# sample_accuracy 为浮点随机数向整数映射时整数的范围，该值越大则保留小数点位越多，越精确，但耗时和空间则越大
# banner_size * thread_pool_size 和运行时占用内存正相关，可以根据实际情况调整，乘积为1e7时约占用11.5G内存
# banner_size * thread_number 为总共进行的采样次数
guarantee_weapon_rate = 0.2
guarantee_stigmata_rate = 0.1
guarantee_4_star_exclude_up_rate = 1 - guarantee_weapon_rate - guarantee_stigmata_rate * 3
guarantee_small = 10
guarantee_big = 50

# 卡池大小建议最小在1e3，否则可能不准确或者出现越界问题
# 新卡池是假设使用了精准的概率和扩充的保底实现的，后续会根据正式服公示消息调整
sample_accuracy = int(1e4)
banner_size = int(1e6)
pnt_threshold = banner_size - int(2e3)
gacha_sample_size = int(1e5)
x_truncate = 200
preserved_size = 1500

# -2 非四星装备  -1 四星非UP装备  0 UP武器  1 UP圣痕上  2 UP圣痕中  3 UP圣痕下
guarantee_banner, focus_regular_banner, expansion_regular_banner, new_regular_banner = {}, {}, {}, {}

In [55]:
def generate_banner():
    pnt, val = 0, -1
    for i in range(sample_accuracy):
        guarantee_banner.update({i: -2})
        focus_regular_banner.update({i: -2})
        expansion_regular_banner.update({i: -2})
        new_regular_banner.update({i: -2})

    for i in np.array(
            [
                guarantee_4_star_exclude_up_rate,
                guarantee_weapon_rate,
                guarantee_stigmata_rate,
                guarantee_stigmata_rate,
                guarantee_stigmata_rate
            ]) * sample_accuracy:
        for j in range(int(i)):
            guarantee_banner[pnt] = val
            pnt += 1
        val += 1

    pnt, val = 0, -1
    for i in np.array(
            [
                focus_regular_4_star_exclude_up_rate,
                focus_regular_weapon_rate,
                focus_regular_stigmata_rate,
                focus_regular_stigmata_rate,
                focus_regular_stigmata_rate
            ]) * sample_accuracy:
        for j in range(int(i)):
            focus_regular_banner[pnt] = val
            pnt += 1
        val += 1

    pnt, val = 0, -1
    for i in np.array(
            [
                expansion_regular_4_star_exclude_up_rate,
                expansion_regular_weapon_rate,
                expansion_regular_stigmata_rate,
                expansion_regular_stigmata_rate,
                expansion_regular_stigmata_rate
            ]) * sample_accuracy:
        for j in range(int(i)):
            expansion_regular_banner[pnt] = val
            pnt += 1
        val += 1

    pnt, val = 0, -1
    for i in np.array(
            [
                new_regular_4_star_exclude_up_rate,
                new_regular_weapon_rate,
                new_regular_stigmata_rate,
                new_regular_stigmata_rate,
                new_regular_stigmata_rate
            ]) * sample_accuracy:
        for j in range(int(i)):
            new_regular_banner[pnt] = val
            pnt += 1
        val += 1


generate_banner()

In [56]:
class Gacha_Focus:
    def __init__(self, args):
        self.args = args
        self.pity_counter_small = 0
        self.pity_counter_big = 0
        self.equipment = np.array([0, 0, 0, 0])
        self.total_pulling = 0

    def single_pull(self):
        result = guarantee_banner[self.args['random_pool'][self.args['pnt_random_pool']]] \
            if self.pity_counter_small >= guarantee_small \
            else focus_regular_banner[self.args['random_pool'][self.args['pnt_random_pool']]]
        self.args['pnt_random_pool'] += 1

        if result >= -1:
            self.pity_counter_small = 0

            if result >= 0:
                self.equipment[result] += 1
                return True

        return False

    def reset_pity_counter(self):
        return True if \
            (0 not in self.equipment or
             (self.equipment[0] and np.count_nonzero(self.equipment) == 3 and
              (self.equipment[1] + self.equipment[2] + self.equipment[3]) >= 4)) \
            else False

    def pull(self):
        while True:
            self.pity_counter_small += 1
            self.total_pulling += 1

            if self.single_pull() and self.reset_pity_counter():
                break

        return [self.total_pulling, self.equipment]

In [57]:
class Gacha_Expansion:
    def __init__(self, args):
        self.args = args
        self.pity_counter_small = 0
        self.pity_counter_big = 0
        self.equipment = np.array([0, 0, 0, 0])
        self.total_pulling = 0

    def single_pull(self):
        if self.pity_counter_big >= guarantee_big:
            self.pity_counter_small = 0
            self.pity_counter_big = 0
            self.equipment[random.sample(list(np.where(self.equipment == 0)[0]), 1)[0]] += 1

            return True
        else:
            result = guarantee_banner[self.args['random_pool'][self.args['pnt_random_pool']]] \
                if self.pity_counter_small >= guarantee_small \
                else expansion_regular_banner[self.args['random_pool'][self.args['pnt_random_pool']]]
            self.args['pnt_random_pool'] += 1

            if result >= -1:
                self.pity_counter_small = 0

                if result >= 0:
                    if self.equipment[result] == 0:
                        self.pity_counter_big = 0

                    self.equipment[result] += 1
                    return True

            return False

    def reset_pity_counter(self):
        return True if \
            (0 not in self.equipment or
             (self.equipment[0] and np.count_nonzero(self.equipment) == 3 and
              (self.equipment[1] + self.equipment[2] + self.equipment[3]) >= 4)) \
            else False

    def pull(self):
        while True:
            self.pity_counter_small += 1
            self.pity_counter_big += 1
            self.total_pulling += 1

            if self.single_pull() and self.reset_pity_counter():
                break

        return [self.total_pulling, self.equipment]

In [58]:
class Gacha_New:
    def __init__(self, args):
        self.args = args
        self.pity_counter_small = 0
        self.pity_counter_big = 0
        self.equipment = np.array([0, 0, 0, 0])
        self.total_pulling = 0

    def single_pull(self):
        if self.pity_counter_big >= guarantee_big:
            self.pity_counter_small = 0
            self.pity_counter_big = 0
            self.equipment[random.sample(list(np.where(self.equipment == 0)[0]), 1)[0]] += 1

            return True
        else:
            result = guarantee_banner[self.args['random_pool'][self.args['pnt_random_pool']]] \
                if self.pity_counter_small >= guarantee_small \
                else new_regular_banner[self.args['random_pool'][self.args['pnt_random_pool']]]
            self.args['pnt_random_pool'] += 1

            if result >= -1:
                self.pity_counter_small = 0

                if result >= 0:
                    if self.equipment[result] == 0:
                        self.pity_counter_big = 0

                    self.equipment[result] += 1
                    return True

            return False

    def reset_pity_counter(self):
        return True if \
            (0 not in self.equipment or
             (self.equipment[0] and np.count_nonzero(self.equipment) == 3 and
              (self.equipment[1] + self.equipment[2] + self.equipment[3]) >= 4)) \
            else False

    def pull(self):
        while True:
            self.pity_counter_small += 1
            self.pity_counter_big += 1
            self.total_pulling += 1

            if self.single_pull() and self.reset_pity_counter():
                break

        return [self.total_pulling, self.equipment]

In [59]:
def test():
    # 如果出现了超过1500的结果，那你可能遇到了绝世非酋
    y_focus_temp, y_expansion_temp, y_new_temp = [0] * preserved_size, [0] * preserved_size, [0] * preserved_size

    args = {
        'random_pool': np.random.randint(0, sample_accuracy, banner_size),
        'pnt_random_pool': 0
    }

    for i in range(gacha_sample_size):
        result_focus = Gacha_Focus(args).pull()[0]
        result_expansion = Gacha_Expansion(args).pull()[0]
        result_new = Gacha_New(args).pull()[0]

        y_focus_temp[result_focus] += 1
        y_expansion_temp[result_expansion] += 1
        y_new_temp[result_new] += 1

        if args['pnt_random_pool'] >= pnt_threshold:
            np.random.shuffle(args['random_pool'])
            args['pnt_random_pool'] = 0

    return [y_focus_temp, y_expansion_temp, y_new_temp]


start_time = time.time()

[y_focus, y_expansion, y_new] = test()

print('总共耗时{}s'.format(int(time.time() - start_time)))

总共耗时40s


In [60]:
def draw_result():
    data = [
        go.Scatter(
            x=[i for i in range(x_truncate)],
            y=y_focus[:x_truncate],
            mode='markers',
            marker={
                'color': 'rgb(255, 0, 0)'
            },
            name='focus'
        ),
        go.Scatter(
            x=[i for i in range(x_truncate)],
            y=y_expansion[:x_truncate],
            mode='markers',
            marker={
                'color': 'rgb(0, 0, 255)'
            },
            name='expansion'
        ),
        go.Scatter(
            x=[i for i in range(x_truncate)],
            y=y_new[:x_truncate],
            mode='markers',
            marker={
                'color': 'rgb(0, 255, 0)'
            },
            name='new'
        ),
    ]
    fig = go.Figure(
        data=data
    )
    offline.iplot(fig)

draw_result()

In [61]:
def generate_statistics():
    # 因为离群点的存在，原始数据中的众数好像不是很有意义，想要了解的可以直接与上图交互
    sum_focus, sum_expansion, sum_new = 0, 0, 0
    num_focus, num_expansion, num_new = sum(y_focus), sum(y_expansion), sum(y_new)

    for i in range(x_truncate):
        sum_focus += i * y_focus[i]
        sum_expansion += i * y_expansion[i]
        sum_new += i * y_expansion[i]

    print('精准补给平均数: {}'.format(sum_focus / num_focus))
    print('扩充补给平均数: {}'.format(sum_expansion / num_expansion))
    print('新补给池平均数: {}'.format(sum_new / num_new))

    temp_num_focus, temp_num_expansion, temp_num_new = num_focus / 2.0, num_expansion / 2.0, num_new / 2.0

    for i in range(x_truncate):
        if temp_num_focus >= y_focus[i]:
            temp_num_focus -= y_focus[i]
        else:
            print('精准补给中位数: {}'.format(i))
            break

    for i in range(x_truncate):
        if temp_num_expansion >= y_expansion[i]:
            temp_num_expansion -= y_expansion[i]
        else:
            print('扩充补给中位数: {}'.format(i))
            break

    for i in range(x_truncate):
        if temp_num_new >= y_new[i]:
            temp_num_new -= y_new[i]
        else:
            print('新补给池中位数: {}'.format(i))
            break

generate_statistics()

精准补给平均数: 98.58257
扩充补给平均数: 99.07852
新补给池平均数: 99.07852
精准补给中位数: 101
扩充补给中位数: 98
新补给池中位数: 91
